<a href="https://colab.research.google.com/github/francescadima1999/Esempio1/blob/main/Tuplet_Extractor_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Installazione dei packages

In [9]:
!python -V  #Python 3.10.12

Python 3.11.12


In [10]:
!nvcc --version # find the CUDA driver build above

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


`cu121`: CUDA 12.1 - `cu122`: CUDA 12.2 - `cu123`: CUDA 12.3 - `cu124`: CUDA 12.4

In [11]:
# Install key libraries for LLM

#Install llama-cpp-python with CUDA per utilizzare la GPU
!set CMAKE_ARGS=-DGGML_CUDA=on
!set FORCE_CMAKE=1

#Install llama-cpp-python, cuda-enabled package
!python -m pip install llama-cpp-python --prefer-binary --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu122 --force-reinstall

#Install pytorch-related, cuda-enabled package
!pip install torch==2.3.0 torchvision torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 48.0 MB/s eta 0:00:00
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
  Att

## Step 2: Download del LLM da hugging face

Use the "hf_hub_download" function to download models on huggingface.


In [12]:
import torch
import huggingface_hub

#####Modello LLAMA

In [8]:
import os
from huggingface_hub import hf_hub_download

# Define the model name and file
model_name = "MaziyarPanahi/Llama-3-8B-Instruct-v0.10-GGUF"

#Quantizzazione 1
#model_file = "Llama-3-8B-Instruct-v0.10.Q4_K_M.gguf" # this is the specific model file we'll use in this example. It's a 4-bit quant, but other levels of quantization are available in the model repo if preferred

#Quantizzazione 2
# model_file = "Llama-3-8B-Instruct-v0.10.Q5_K_M.gguf"

#Quantizzazione 3
model_file = "Llama-3-8B-Instruct-v0.10.Q8_0.gguf"


# Download the model from Hugging Face Hub
model_path = hf_hub_download(
    model_name,
    filename=model_file,
    local_dir='models/',  # Download the model to the "models" folder
    )
#Copia il path nel prossimo comando
print("My model path:", model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Llama-3-8B-Instruct-v0.10.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

My model path: models/Llama-3-8B-Instruct-v0.10.Q8_0.gguf


In [ ]:
del llm #Ripulisci la gpu dai modelli caricati

Note that BLAS = 1 means GPU is enabled:
*   AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 |



## Step 3: Utilizzo del LLM

####Import del dataset

In [ ]:
import pandas as pd

df = pd.read_excel('/content/DisinformNews Dataset.xlsx')
print(df.head(3))

   ID                                                URL  \
0   1  https://web.archive.org/web/20161011224653/htt...   
1   2  https://web.archive.org/web/20161015002001/htt...   
2   3  https://web.archive.org/web/20160928083422/htt...   

                                              TITOLO               SOURCE  \
0  Pope Francis Shocks World, Endorses Donald Tru...  Ending the Fed News   
1  IT’S OVER: Hillary’s ISIS Email Just Leaked & ...  Ending the Fed News   
2  Just Read the Law: Hillary is Disqualified fro...  Ending the Fed News   

                                               TESTO  \
0  VATICAN CITY – News outlets around the world a...   
1  Today Wikileaks released what is, by far, the ...   
2  We’re so used to complicated problems that we ...   

                   CAMPAGNA THREAT ACTOR TIPO  
0  Viral Fake Election News          NaN  WEB  
1  Viral Fake Election News          NaN  WEB  
2  Viral Fake Election News          NaN  WEB  


**Separazione delle colonne del dataset**

In [ ]:
# URL , TITOLO, SOURCE, TESTO, CAMPAGNA , THREAT ACTOR, TIPO
colonna_id= df['ID']
colonna_url = df['URL']
colonna_titolo = df['TITOLO']
colonna_source = df['SOURCE']
colonna_testo = df['TESTO']
colonna_campagna= df['CAMPAGNA']
colonna_threat_actor = df['THREAT ACTOR']
colonna_tipo = df['TIPO']

In [ ]:
print(colonna_testo[0])
print(len(colonna_testo[0]))

VATICAN CITY – News outlets around the world are reporting on the news that Pope Francis has made the unprecedented decision to endorse a US presidential candidate. His statement in support of Donald Trump was released from the Vatican this evening: “I have been hesitant to offer any kind of support for either candidate in the US presidential election but I now feel that to not voice my concern would be a dereliction of my duty as the Holy See. A strong and free America is vitally important in maintaining a strong and free world and in that sense what happens in American elections affects us all. The Rule of Law is the backbone of the American government as it is in any nation that strives for freedom and I now fear that the Rule of Law in America has been dealt a dangerous blow. The FBI, in refusing to recommend prosecution after admitting that the law had been broken on multiple occasions by Secretary Clinton, has exposed itself as corrupted by political forces that have become far t

####Setup temperature

In [ ]:
#torch.cuda.empty_cache() # Clear GPU cache

**Run Temperatura=0**

In [ ]:
def model_run(user_input):
    # Definisci la sequenza di stop
    stop_sequence = ["##END LIST##","#END LIST#","END LIST"]  # Sostituisci con la sequenza desiderata

    # Crea la richiesta di completamento
    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a bot that extracts representative elements of its high-level objective from the text"},
            {"role": "user", "content": user_input}
        ],
        temperature=0,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']


**Run Temperatura = 0.3**

In [ ]:
def model_run(user_input):
    # Definisci la sequenza di stop
    stop_sequence = ["##END LIST##","#END LIST#","END LIST"]  # Sostituisci con la sequenza desiderata

    # Crea la richiesta di completamento
    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a bot that extracts representative elements of its high-level objective from the text"},
            {"role": "user", "content": user_input}
        ],
        temperature=0.3,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']

**Run Temperatura = 0.6**

In [ ]:
def model_run(user_input):
    # Definisci la sequenza di stop
    stop_sequence = ["##END LIST##","#END LIST#","END LIST"]  # Sostituisci con la sequenza desiderata

    # Crea la richiesta di completamento
    output = llm.create_chat_completion(
        messages= [
            {"role": "system", "content": "You are a bot that extracts representative elements of its high-level objective from the text"},
            {"role": "user", "content": user_input}
        ],
        temperature=0.6,
        top_p=0.95,
        stop=stop_sequence
    )

    return output['choices'][0]['message']['content']

####Funzione di estrazione dal testo

In [ ]:
import re
def extract_tuples(text):
    # Trova la parte del testo prima di "##END LIST## (1)"
    before_note = text.split("##END LIST## (1)")[0]

    # Regex per trovare le tuple nel formato "soggetto - verbo - oggetto"
    # Ignora il numero iniziale seguito da un punto o altri caratteri
    pattern = r'\d*\.*\s*(.*? - .*? - .*?)\n'

    # Trova tutte le tuple
    matches = re.findall(pattern, before_note)

    return matches

####Debug singolo run

In [ ]:
user_input = f'''Read the following text and identify all the tuples in the subject-verb-object form. The tuples should reflect the main actions and relationships between the entities mentioned in the text. Follow these steps:

              Identify the subject of the action.
              Identify the verb that describes the action or relationship.
              Identify the object or destination of the action.
              Return the tuples in this format: Subject - Verb - Object.


              Example:
              Text: 'John gave a book to Mary.'
              Tuple: 'John - gave - a book to Mary'
              ##END LIST##

              Apply this method to the following text:, {colonna_testo[9]} '''


# Invia il messaggio e stampa la risposta
response = model_run(user_input)
print(response)

Llama.generate: prefix-match hit


 <<SYS>> </s>
<s>[INST] <<SYS>> 
Extract the main actions and relationships between entities mentioned in the text. Represent these as subject-verb-object (SVO) tuples.

1. Paul Horner - was paid - $3,500
2. Paul Horner - answered - a Craigslist ad
3. Paul Horner - interviewed - with them
4. Paul Horner - got the part 
5. Paul Horner - received - an actual check from 'Women Are The Future'
6. Paul Horner - was told - if anyone asked, he should talk about Bernie Sanders
7. Tom Downey - knew - those weren’t real protesters
8. Tom Downey - shouted - at them the whole time
9. Tom Downey - got high-fives - from his fellow Trump supporters
10. Paul Horner - saw most of the people - during the interview and training for the rally
11. Paul Horner - learned - they only paid Latinos $500, Muslims $600, African Americans $750, and illegals received $300 across the board
12. Sarah Bradley - said she would protest Trump for free 
13. Sarah Bradley - thinks Trump supporters are the last kind of peop

In [ ]:
tuples = extract_tuples(response)
print(tuples)

["James Comey - became - a Clinton Foundation defense contractor's recipient of millions of dollars", "James Comey - served - on a Clinton Foundation corporate partner's board", 'James Comey - had - a financial relationship with his brother Peter Comey', 'James Comey - promised - to recuse himself on all cases involving former employers', 'James Comey - earned - $6 million in one year from Lockheed Martin', 'Lockheed Martin - became - a Clinton Foundation donor that very year', 'James Comey - left - the Bush administration and went to Lockheed Martin', 'Lockheed Martin - admitted - to becoming a Clinton Global Initiative member in 2010', "Lockheed Martin - won - 17 approvals for private contracts from Hillary Clinton's State Department in 2010", 'HSBC Holdings - partnered - with Deutsche Bank through the Clinton Foundation', 'James Comey - became - a board member, director, and Financial System Vulnerabilities Committee member of HSBC Holdings', 'James Comey - was not questioned - abou

####Run Iterato

In [ ]:
import time
import pandas as pd
import re

#print(content)

# Calculate runtime
#runtime = end_time - start_time
#print("response run time is: ", runtime)

# Start the timer
start_time = time.time()
for i in range(10):

  torch.cuda.empty_cache() # Clear GPU cache
  user_input = f'''Read the following text and identify all the tuples in the subject-verb-object form. The tuples should reflect the main actions and relationships between the entities mentioned in the text. Follow these steps:

              Identify the subject of the action.
              Identify the verb that describes the action or relationship.
              Identify the object or destination of the action.
              Return the tuples in this format: Subject - Verb - Object.
              Print "END LIST" at the end of the list.

              Example:
              Text: 'John gave a book to Mary.'
              Tuple: 'John - gave - a book to Mary'
              ##END LIST##

              Apply this method to the following text:, {colonna_testo[i]} '''


  # Invia il messaggio e stampa la risposta
  response = model_run(user_input)
  print(response)
  print("-----------------------FINE--------------------------------------")
  # Estrazione delle tuple complete
  tuples = extract_tuples(response)

  if i == 0:
    # Creare un DataFrame
    df_tuple = pd.DataFrame(tuples, columns=['TUPLA'])
     # Aggiungere la colonna 'ID' con il valore di colonna_ID[0]
    df_tuple['ID ARTICOLO'] = colonna_id[i]
    df_tuple['CAMPAGNA'] = colonna_campagna[i]
  else:
    # Create temporary DataFrames for new data
    df_temp = pd.DataFrame({'TUPLA': tuples, 'ID ARTICOLO': colonna_id[i], 'CAMPAGNA': colonna_campagna[i]})
    # Concatenate the temporary DataFrame with the main DataFrame
    df_tuple = pd.concat([df_tuple, df_temp], ignore_index=True)

# Stop the timer
end_time = time.time()

# Calculate runtime
runtime = end_time - start_time
print("response run time is: ", runtime)
print(df_tuple.head(50))

Llama.generate: prefix-match hit


 >>>>

The tuples of main actions and relationships between entities are:

Pope Francis - endorsed - Donald Trump
Pope Francis - voiced - concern
Pope Francis - felt - a dereliction of duty
Pope Francis - expressed - opinion on American elections
Pope Francis - stated - importance of strong America for world
Pope Francis - mentioned - Rule of Law in America
Pope Francis - criticized - FBI's refusal to recommend prosecution
Pope Francis - criticized - corruption in the government
Pope Francis - asked - Americans to vote for Donald Trump


-----------------------FINE--------------------------------------


Llama.generate: prefix-match hit


 <<SYS>>


Here's a list of tuples:

Hillary - was the subject of - Trump's dirty talk video
Trump - called - Hillary 'the founder' of ISIS
media - ripped apart - Trump over his statement about Hillary being the 'founder' of ISIS
Assange - promised - latest batch of leaks would lead to the indictment of Hillary
Wikileaks - released - a batch of leaked emails
Hillary - admitted in her own words - to funding and running ISIS
John Podesta - was the recipient of - an email from 2014, which was released today
media - has not reported on - the content of the latest Wikileaks release yet
Wikileaks - has been accurate in their leaks for a decade without releasing any false information
Saudi Arabia and Qatar - funded 20% of - Hillary's Presidential campaign
Clinton Foundation - received funding from - Saudi Arabia, Qatar, and others who are also funding ISIS
Hillary - was complicit in the funding and arming of ISIS by our allies, Saudi Arabia and Qatar
United States - created, armed, and funded

Llama.generate: prefix-match hit


...

 <<SYS>>>
You are a bot that extracts representative elements of its high-level objective from the text
<</SYS>> 
Extracting main actions, relationships, and entities:
- Randall Prince - found - approximately one dozen black, sealed ballot boxes filled with thousands of Franklin County votes for Hillary Clinton and other Democrat candidates
- Randall Prince - went - to investigate
- Randall Prince - saw - several black boxes in an otherwise empty room
- Election officials in Franklin County, Ohio - are - reportedly stumped over what Randall Prince found 
- Randall Prince - is - a Trump supporter
- The Clinton campaign - likely goal was - to slip the fake ballot boxes in with the real ballot boxes when they went to official election judges on November 8th
- Many national Democrat groups - have pulled funding from Ohio, redirecting it to places where they are doing better
- Hillary Clinton - has spent less time and money in recent weeks in Ohio as it appeared Trump will carry the state
-

Llama.generate: prefix-match hit


 <<SYS>>

Here is the list of tuples in subject-verb-object form:

Barack Obama - has - sensationally told
Barack Obama - will NOT - vacate
Donald J. Trump - is - elected
Barack Obama - claims - he is fully prepared
Barack Obama - found - a little known loophole
Barack Obama - believes - is in the best interests
Barack Obama - must - do what he feel
Barack Obama - will be - forced to take
Barack Obama - explained - As president I must
Barack Obama - said - I am not standing down
Barack Obama - responded - I am prepared
Barack Obama - confessed - I am willing to file a motion of no confidence
Blitzer - asked - if he would remain in charge
Obama - responded - Yes if necessary
Obama - cannot - allow him into the chair
Obama - suggested - will barricade himself and his family inside the White House
Secret Service - are instructed by Obama - use full force in defending the White House
Joe - expressed willingness to die multiple times 
Howard Stern - questioned Trump - Don’t worry we’ll get 

Llama.generate: prefix-match hit


 <<SYS>> 
<</SYS>> 
Now, extract representative tuples from the text and return them as per the instructions provided. 

Here are the extracted tuples in the subject-verb-object format:

1. Paul Horner - was - paid
2. Paul Horner - answered - a Craigslist ad
3. Paul Horner - interviewed - with them
4. Paul Horner - got - the part
5. Paul Horner - received - a check
6. Paul Horner - was told - if anyone asked, he should talk about Bernie Sanders
7. Paul Horner - communicated - with people who had AOL email addresses
8. Tom Downey - knew - there was something up 
9. Tom Downey - shouted - at the protesters
10. Tom Downey - got - high-fives from fellow Trump supporters
11. Horner - saw - most of the protesters during interview and training
12. Horner - learned - some people were paid less based on race, gender, or nationality 
13. Sarah Bradley - said - she would protest for free
14. Sarah Bradley - told - Trump is creating a place for hate-filled individuals to gather
15. Paul Horner - w

In [ ]:
# Salvare il DataFrame in un file Excel
df_tuple.to_excel('TuplesExtracted_Q8_06TEMP.xlsx', index=False)
print(runtime)
print("File Excel creato con successo!")

388.83690667152405
File Excel creato con successo!


In [ ]:
RT_Q5KM_0TEMP=371.50677728652954
RT_Q5KM_03TEMP=385.1130666732788
RT_Q5KM_06TEMP=394.9632875919342
RT_Q8_0TEMP=340.24108028411865
RT_Q8_03TEMP= 390.331839799881
RT_Q8_06TEMP= 388.83690667152405